In [57]:
import nbformat as nbf
import textwrap

In [58]:
nb = nbf.v4.new_notebook()

In [59]:
#this creates a button to toggle to remove code in html
source_1 = textwrap.dedent("""
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [60]:
#import libraries
source_1 = textwrap.dedent("""\
#import libraries
import warnings 
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
import math 
import re 
import pandas as pd
import os
import glob
from os.path import expanduser
import datetime
""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [61]:
#define functions
source_1 = textwrap.dedent("""\
def color_negative_red(val):
    color = 'red' if val > 110 else 'black'
    return 'color: %s' % color
""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

source_1 = textwrap.dedent("""\
def compute_rom(met,k,invT,nkeypoints):    
    #joint we want to evaluate in the current session
    jnt = getattr(k,met.ref_joint)
    jsess = jnt
    
    #direction wrt we want to evaluate the metric
    dirsess = met.ref_direction

    #joint child
    jskel = getattr(skel,met.ref_joint)

    cld = jskel.child
    childval = getattr(k,cld[0])
    child = childval
    
    #compute metric
    x=jsess[:,0]
    y=jsess[:,1]
    z=jsess[:,2]

    xchild=child[:,0]
    ychild=child[:,1]
    zchild=child[:,2]

    v1x = xchild-x
    v1y = ychild-y
    v1z = zchild-z
    v1=np.array([v1x,v1y,v1z])
    v1=np.transpose(v1)

    #plane over which we want to evaluate the metric
    planesess = met.ref_plane
    
    #project v1 on the right plane
    cosRom = []
    for i in range(0,nkeypoints):
        temp_ref = np.array([x[i],y[i],z[i],1])
        temp_child = np.array([xchild[i],ychild[i],zchild[i],1])
    
        transf_ref = np.inner(invT,temp_ref) #invT.dot(temp_ref)
        transf_child = np.inner(invT,temp_child) # invT.dot(temp_child)
        vprocess = transf_child-transf_ref
        vprocess = np.delete(vprocess,3)

        dist = np.dot(vprocess,np.transpose(planesess[i]))   
        vprocess = vprocess-dist*planesess[i]
    
        n1 = np.linalg.norm(vprocess)
        if(n1>0):
            vprocess = vprocess/n1
    
        n2 = np.linalg.norm(dirsess)
        if(n2>0):
            dirsess = dirsess/n2
    
        dotprod = np.dot(vprocess,np.transpose(dirsess))
        cosRom.append(dotprod)

    rom_value = np.arccos(cosRom)
    result = rom_value *(180/math.pi)
    
    return result
""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

source_1 = textwrap.dedent("""\
def compute_ep(met,k,invT,nkeypoints,planesess,j1,j2):    
    #joint we want to evaluate in the current session
    jnt = getattr(k,met.ref_joint)
    #print jnt
    jsess = jnt
    x=jsess[:,0]
    y=jsess[:,1]
    z=jsess[:,2]
    
    v1=np.array([x,y,z])
    v1=np.transpose(v1)
            
    #project v1 on the right plane
    resultx = []
    resulty = []
    for i in range(0,nkeypoints):
        temp_jnt = np.array([x[i],y[i],z[i],1])   
        transf_jnt = np.inner(invT,temp_jnt) #invT.dot(temp_ref)
        vprocess = np.delete(transf_jnt,3)
        dist = np.dot(vprocess,np.transpose(planesess))      
        vprocess = vprocess-dist*planesess
        resultx.append(vprocess[j1])
        resulty.append(vprocess[j2])
        
    return [resultx,resulty]
""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [62]:
#print personal data
text = """
## Personal data """
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

source_1 = textwrap.dedent("""  
#load file 
home = expanduser("~")
pth = home + '/.local/share/yarp/contexts/motionAnalyzer'
files = glob.glob(os.path.join(pth, '*.mat'))
lastfile = max(files, key=os.path.getctime)

#print personal data
i = [pos for pos, char in enumerate(lastfile) if char == "-"]
i1 = i[0]
i2 = i[1]
name = lastfile[i1+1:i2]
surname = ""
age = ""

personaldata = []
personaldata.append(name)
personaldata.append(surname)
personaldata.append(age)        
table = pd.DataFrame(personaldata) 
table.rename(index={0:"Name",1:"Surname",2:"Age"},  columns={0:"Patient"}, inplace=True)
table.style """)
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [63]:
#we load all the files with the same user name
source_1 = textwrap.dedent("""  
data = []
ctime = []
filename = []
files.sort(key=os.path.getctime)
for fi in files:
    i = [pos for pos, char in enumerate(lastfile) if char == "-"]
    i1 = i[0]
    i2 = i[1]
    namei = lastfile[i1+1:i2]
    if namei == name:
        filename.append(fi)
        data.append(scipy.io.loadmat(fi))
        ctime.append(os.path.getctime(fi)) """)
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [64]:
#define keypoints
source_1 = textwrap.dedent("""\
    tagkeypoints = []
    tagkeypoints.append('shoulderCenter')
    tagkeypoints.append('head')
    tagkeypoints.append('shoulderLeft')
    tagkeypoints.append('elbowLeft')
    tagkeypoints.append('handLeft')
    tagkeypoints.append('shoulderRight')
    tagkeypoints.append('elbowRight')
    tagkeypoints.append('handRight')
    tagkeypoints.append('hipLeft')
    tagkeypoints.append('kneeLeft')
    tagkeypoints.append('ankleLeft')
    tagkeypoints.append('hipRight')
    tagkeypoints.append('kneeRight')
    tagkeypoints.append('ankleRight') """)
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [65]:
#create skeleton data structure
source_1 = textwrap.dedent("""\
skeleton = namedtuple('skeleton', 
                      ['shoulderCenter','head','shoulderLeft','shoulderRight',
                      'elbowLeft','handLeft','elbowRight','handRight',
                      'hipLeft','hipRight','ankleLeft','ankleRight',
                      'kneeLeft','kneeRight'])

shoulderCenter = namedtuple('shoulderCenter', ['parent','child'])
head = namedtuple('head', ['parent','child'])
shoulderLeft = namedtuple('shoulderLeft', ['parent','child'])
shoulderRight = namedtuple('shoulderRight', ['parent','child'])
elbowLeft = namedtuple('elbowLeft', ['parent','child'])
handLeft = namedtuple('handLeft', ['parent','child'])
elbowRight = namedtuple('elbowRight', ['parent','child'])
handRight = namedtuple('handRight', ['parent','child'])
hipLeft = namedtuple('hipLeft', ['parent','child'])
hipRight = namedtuple('hipRight', ['parent','child'])
ankleLeft = namedtuple('ankleLeft', ['parent','child'])
ankleRight = namedtuple('ankleRight', ['parent','child'])
kneeLeft = namedtuple('kneeLeft', ['parent','child'])
kneeRight = namedtuple('kneeRight', ['parent','child'])

scSubst = shoulderCenter([''],['head','shoulderLeft','shoulderRight','hipLeft','hipRight'])
heSubst = head(['shoulderCenter'],[''])
slSubst = shoulderLeft(['shoulderCenter'],['elbowLeft'])
srSubst = shoulderRight(['shoulderCenter'],['elbowRight'])
elSubst = elbowLeft(['shoulderLeft'],['handLeft'])
hlSubst = handLeft(['elbowLeft'],[''])
erSubst = elbowRight(['shoulderRight'],['handRight'])
hrSubst = handRight(['elbowRight'],[''])
hilSubst = hipLeft(['shoulderCenter'],['kneeLeft'])
hirSubst = hipRight(['shoulderCenter'],['kneeRight'])
alSubst = ankleLeft(['kneeLeft'],[''])
arSubst = ankleRight(['kneeRight'],[''])
klSubst = kneeLeft(['hipLeft'],['ankleLeft'])
krSubst = kneeRight(['hipRight'],['ankleRight'])

skel = skeleton(scSubst,heSubst,slSubst,srSubst,elSubst,hlSubst,erSubst,
                  hrSubst,hilSubst,hirSubst,alSubst,arSubst,klSubst,krSubst)

""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [66]:
source_1 = textwrap.dedent("""\
mini_rom = []
maxi_rom = []
tex_rom = []
ti_rom = []
tex_ep = []
ti_ep = []
result = []
resultx_sagittal = []
resulty_sagittal = []
resultx_frontal = []
resulty_frontal = []
xtraj = []
ytraj = []
ztraj = []
tottime = []
metric_tag1 = "ROM"
metric_tag2 = "EP"
for i in range(0,len(data)):
    
    datai = data[i]
        
    keypoints = namedtuple('keypoints', 
                      ['shoulderCenter','head','shoulderLeft','shoulderRight',
                      'elbowLeft','handLeft','elbowRight','handRight',
                      'hipLeft','hipRight','ankleLeft','ankleRight','kneeLeft','kneeRight'])

    time = datai['Time_samples']
    shoulderCenter = datai['Keypoints']['shoulderCenter'][0,0]
    head = datai['Keypoints']['head'][0,0]
    shoulderLeft = datai['Keypoints']['shoulderLeft'][0,0]
    shoulderRight = datai['Keypoints']['shoulderRight'][0,0]
    elbowLeft = datai['Keypoints']['elbowLeft'][0,0]
    handLeft = datai['Keypoints']['handLeft'][0,0]
    elbowRight = datai['Keypoints']['elbowRight'][0,0]
    handRight = datai['Keypoints']['handRight'][0,0]
    hipLeft = datai['Keypoints']['hipLeft'][0,0]
    hipRight = datai['Keypoints']['hipRight'][0,0]
    ankleLeft = datai['Keypoints']['ankleLeft'][0,0]
    ankleRight = datai['Keypoints']['ankleRight'][0,0]
    kneeLeft = datai['Keypoints']['kneeLeft'][0,0]
    kneeRight = datai['Keypoints']['kneeRight'][0,0]

    k = keypoints(shoulderCenter,head,shoulderLeft,shoulderRight,
        elbowLeft,handLeft,elbowRight,handRight,
        hipLeft,hipRight,ankleLeft,ankleRight,kneeLeft,kneeRight)   
    
    metric_tagi = ""
    if metric_tag1 in datai.keys():
        metric_tagi = metric_tag1
    if metric_tag2 in datai.keys():
        metric_tagi = metric_tag2
               
    metrici = namedtuple(metric_tagi, ['motion_type','ref_joint','ref_direction','ref_plane','max','min','tstart','tend'])
    motion_type = datai[metric_tagi]['motion_type'][0,0].tostring()
    motion_type = re.sub(r'[^\w]', '',motion_type)
    refjoint = datai[metric_tagi]['ref_joint'][0,0].tostring()
    refjoint = re.sub(r'[^\w]', '',refjoint)
    refdir = datai[metric_tagi]['ref_direction'][0,0]
    refplane = datai[metric_tagi]['ref_plane'][0,0]
    maxv = datai[metric_tagi]['max'][0,0]
    minv = datai[metric_tagi]['min'][0,0]
    tstart = datai[metric_tagi]['tstart'][0,0]
    tend = datai[metric_tagi]['tend'][0,0]
    met = metrici(motion_type,refjoint,refdir,refplane,maxv,minv,tstart,tend)
    
    nkeypoints = len(shoulderCenter)

    sagittal = k.shoulderLeft[0]-k.shoulderRight[0]
    sagittal = sagittal/np.linalg.norm(sagittal)
    transverse = k.shoulderCenter[0]-0.5*(k.hipLeft[0]+k.hipRight[0])
    transverse = transverse/np.linalg.norm(transverse)
    coronal = np.cross(sagittal,transverse)
    pSC = k.shoulderCenter[0]

    T = np.zeros((4, 4))
    T[0,0]=coronal[0]
    T[1,0]=coronal[1]
    T[2,0]=coronal[2]
    T[0,1]=sagittal[0]
    T[1,1]=sagittal[1]
    T[2,1]=sagittal[2]
    T[0,2]=transverse[0]
    T[1,2]=transverse[1]
    T[2,2]=transverse[2]
    T[0,3]=pSC[0]
    T[1,3]=pSC[1]
    T[2,3]=pSC[2]
    T[3,3]=1
    invT = np.linalg.inv(T)
    
    temp = []
    tempx = []
    tempy = []
    if metric_tagi == "ROM":
        temp = compute_rom(met,k,invT,nkeypoints)
        mini_rom.append(np.asscalar(min(temp)))
        maxi_rom.append(np.asscalar(max(temp)))
        tex_rom.append(np.asscalar(met.tend-met.tstart))
        ti_rom.append(datetime.datetime.fromtimestamp(ctime[i]))
        tottime.append(time)
        result.append(temp)
        
    if metric_tagi == "EP":
        plane_sagittal = np.array([0.0,1.0,0.0])
        plane_frontal = np.array([1.0,0.0,0.0])
        tempx,tempy = compute_ep(met,k,invT,nkeypoints,plane_sagittal,0,2)
        resultx_frontal.append(tempx)
        resulty_frontal.append(tempy)
        tempx,tempy = compute_ep(met,k,invT,nkeypoints,plane_frontal,1,2)           
        resultx_sagittal.append(tempx)
        resulty_sagittal.append(tempy)
        tex_ep.append(np.asscalar(met.tend-met.tstart))
        ti_ep.append(datetime.datetime.fromtimestamp(ctime[i]))
        tottime.append(time)
        
        jsess = getattr(k,met.ref_joint)
        xtraj.append(jsess[:,0])
        ytraj.append(jsess[:,1])
        ztraj.append(jsess[:,2])

    metric_tag_last = metric_tagi
    maxv_last = maxv
    lastmet = met
""")
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [67]:
#print information about the performed movement and the evaluated metric
text = """ 
## Daily session report
The patient did the following exercise: """
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

source_1 = textwrap.dedent("""  
lastmet.motion_type """)
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

text = """ 
on: """
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

source_1 = textwrap.dedent("""  
now = datetime.datetime.now()
print now """)
code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [68]:
#plot the metrics for each session
text =  """ 
The following shows the metric trend during the exercise:"""
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

source_1 = textwrap.dedent("""\
%matplotlib inline
if metric_tag_last=="ROM":
    plt.figure(figsize=(7,5))
    plt.plot(tottime[-1],result[-1],'b')
    plt.plot(tottime[-1],np.transpose(maxv_last*np.ones(len(tottime[-1]))),'r')
    #plt.yticks(np.arange(0,180, step=30))
    plt.xlabel('Time [s]')
    plt.ylabel(metric_tag_last)
    plt.legend(['Real trend','Ideal trend'],bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    plt.show()
if metric_tag_last=="EP":
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.plot3D(xtraj[-1], ytraj[-1], ztraj[-1],'b')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')
    ax.set_zlabel('z [cm]')
    plt.title("End-point trajectory")
    plt.show()      """)

code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [69]:
#report about patient's improvements
text =  """ 
## Patient progress"""
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

text =  """ 
The trends of the patient metrics, grouped by month, are shown here: """
markdown_cell = nbf.v4.new_markdown_cell(text)
nb.cells.append(markdown_cell)

source_1 = textwrap.dedent("""\
if ti_rom:
    frame = pd.DataFrame({'t':ti_rom,'max':maxi_rom, 'min':mini_rom})
    ax = (frame.set_index('t'). # use date-time as index
     assign(Month=lambda t: t.index.month). # add new column with month
     groupby('Month'). # group by that column
     mean(). # find a sum of the only column 'y'
     plot.bar(title="Range of Motion Parameters",figsize=(7,5),color=['b','r'])); # make a barplot
    ax.set_ylabel("Degrees")
    
if ti_ep:
    frame = pd.DataFrame({'t':ti_ep,'tex':tex_ep})
    ax = (frame.set_index('t'). # use date-time as index
     assign(Month=lambda t: t.index.month). # add new column with month
     groupby('Month'). # group by that column
     mean(). # find a sum of the only column 'y'
     plot.bar(title="End Point Parameters",figsize=(7,5),color=['g'])); # make a barplot
    ax.set_ylabel("Seconds") """)

code_cell = nbf.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell)

In [70]:
nbf.write(nb, 'report-eng.ipynb')